In [2]:
import numpy as np
import pandas as pd

import xlrd
import time
import xlsxwriter as xlsxwriter
import csv
import math
from datetime import datetime
import re

In [332]:
df = pd.read_csv("test_crawl_total.csv", thousands = ",", encoding = "utf-8")

In [333]:
df = df[["영화명","개봉일","누적매출액", "누적관객수", "스크린수","상영횟수","대표국적", "배급사", "등급","장르","감독","배우"]]

## 1. 장르

In [334]:
df["장르"].value_counts()

드라마                598
멜로/로맨스             311
멜로/로맨스,드라마         291
기타                 185
성인물(에로)            140
                  ... 
다큐멘터리,미스터리           1
어드벤처,전쟁,드라마          1
SF,어드벤처              1
드라마,가족,멜로/로맨스        1
드라마,미스터리,사극,스릴러      1
Name: 장르, Length: 365, dtype: int64

In [335]:
for i in range(len(df["장르"])):
    if type(df["장르"][i]) == float:
        df["장르"][i] = "장르없음"
    else:
        df["장르"][i] = df["장르"][i].split(",")[0]

<ipython-input-335-ce42fc033b6c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["장르"][i] = df["장르"][i].split(",")[0]
<ipython-input-335-ce42fc033b6c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["장르"][i] = "장르없음"


In [336]:
df.replace("성인물(에로)", "에로", inplace = True)
df.replace("서부극(웨스턴)", "서부", inplace = True)
df.replace("공연실황", "공연", inplace = True)
df.replace("서스펜스", "스릴러", inplace = True)
df.replace("공포(호러)", "공포", inplace = True)
df.replace("멜로/로맨스", "멜로로맨스", inplace = True)
df.replace("실험", "다큐멘터리", inplace = True)

In [337]:
etc_index = df[df["장르"]=="기타"].index
df.drop(etc_index, inplace=True)

## 2. 등급

In [338]:
df['등급'] = df['등급'].fillna("등급없음")

In [339]:
df.replace([" 청소년 관람불가 ","청소년 관람불가 ","청소년 관람불가","[국내] 청소년 관람불가","고등학생이상관람가,청소년관람불가","청소년관람불가,12세관람가","국민학생관람불가,청소년관람불가","청소년관람불가,고등학생이상관람가","18세관람가,청소년관람불가","18세 미만인 자는 관람할 수 없는 등급","미성년자관람불가",'18세관람가', "연소자관람불가","연소자관람불가,청소년관람불가"], '청소년관람불가',inplace=True)
df.replace([" 15세 관람가 ","15세 관람가 ","[국내] 15세 관람가"," 15세 관람가","15세관람가","고등학생이상관람가","고등학생이상관람가,15세이상관람가","15세 미만인 자는 관람할 수 없는 등급","15세관람가,15세이상관람가","15세이상관람가,중학생이상관람가","15세이상관람가,미성년자관람불가","청소년관람불가,15세이상관람가","연소자관람불가,15세이상관람가","국민학생관람불가,15세이상관람가","15세이상관람가,전체관람가","15세이상관람가,18세 미만인 자는 관람할 수 없는 등급","연소자관람가,15세이상관람가","18세관람가,15세이상관람가","15세 미만인 자는 관람할 수 없는 등급 ,15세이상관람가"], '15세이상관람가',inplace=True)
df.replace([" 12세 관람가 ","12세 관람가 "," 12세 관람가","[국내] 12세 관람가","12세관람가","중학생이상관람가","국민학생관람불가","12세이상관람가,12세관람가","12세 미만인 자는 관람할 수 없는 등급","12세이상관람가,중학생이상관람가","12세이상관람가,국민학생관람불가","12세이상관람가,전체관람가","12세이상관람가,15세 미만인 자는 관람할 수 없는 등급","12세이상관람가,15세이상관람가","12세이상관람가,고등학생이상관람가","12세이상관람가,국민학생관람불가,15세이상관람가","12세이상관람가,연소자관람가","15세관람가,12세이상관람가"],"12세이상관람가", inplace=True)
df.replace(["전체관람가,중학생이상관람가"," 전체 관람가 ","전체 관람가 ","[국내] 전체 관람가","연소자관람가","모든 관람객이 관람할 수 있는 등급","연소자관람가,전체관람가","청소년관람불가,전체관람가","미성년자관람가","모든 관람객이 관람할 수 있는 등급,전체관람가","12세이상관람가,연소자관람가,전체관람가"],"전체관람가",inplace=True)
df.replace(["미정"], "등급없음", inplace = True)

In [340]:
df["등급"].value_counts()

청소년관람불가     996
등급없음        681
15세이상관람가    509
12세이상관람가    396
전체관람가       248
Name: 등급, dtype: int64

## 3. 대표국적

In [341]:
nation = ["미국", "한국", "일본", "프랑스", "영국", "독일", "중국", "이탈리아", "캐나다", "스페인"]

for i in range(len(df)):
    if df.iloc[i]["대표국적"] not in nation:
        df["대표국적"].replace(df.iloc[i]["대표국적"],"기타",inplace=True)

In [342]:
df["대표국적"].value_counts()

한국      909
미국      688
일본      563
기타      240
프랑스     172
영국       83
이탈리아     57
중국       51
독일       36
캐나다      17
스페인      14
Name: 대표국적, dtype: int64

## 4. 배우 / 감독 / 배급사

In [343]:
df.reset_index(inplace = True)
df.drop("index", axis=1, inplace = True)

In [344]:
# 배우 3명 남기기

for i in range(len(df["배우"])):
    if type(df["배우"][i]) == float:
        df["배우"][i] = 0
    else:
        List = str(df["배우"][i]).split(" ")
        df["배우"][i] = "".join(List)
        
for i in range(len(df["배우"])):
    if type(df["배우"][i]) == float:
        df["배우"][i] = 0
    else:
        df["배우"][i] = str(df["배우"][i]).split(",")[0:3]

<ipython-input-344-1221f948d6c6>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["배우"][i] = "".join(List)
<ipython-input-344-1221f948d6c6>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["배우"][i] = 0
<ipython-input-344-1221f948d6c6>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["배우"][i] = str(df["배우"][i]).split(",")[0:3]


In [346]:
# 배급사 1개 남기기

for i in range(len(df["배급사"])):
    if type(df["배급사"][i]) == float:
        df["배급사"][i] = 0
    else:
        df["배급사"][i] = str(df["배급사"][i]).split(",")[0]
        
for i in range(len(df["배급사"])):
    if df["배급사"][i] == 0:
        df["배급사"][i] = 0
    else:
        List = df["배급사"][i].split(" ")
        df["배급사"][i] = "".join(List)

<ipython-input-346-4ee9df4b9ec4>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["배급사"][i] = str(df["배급사"][i]).split(",")[0]
<ipython-input-346-4ee9df4b9ec4>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["배급사"][i] = 0
<ipython-input-346-4ee9df4b9ec4>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["배급사"][i] = "".join(List)
<ipython-input-346-4ee9df4b9ec4>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy

In [348]:
# 감독 1명 남기기

for i in range(len(df["감독"])):
    if type(df["감독"][i]) == float:
        df["감독"][i] = 0
    else:
        df["감독"][i] = str(df["감독"][i]).split(",")[0]
        
for i in range(len(df["감독"])):
    if df["감독"][i] == 0:
        df["감독"][i] = 0
    else:
        List = df["감독"][i].split(" ")
        df["감독"][i] = "".join(List)

<ipython-input-348-498ff371356a>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["감독"][i] = str(df["감독"][i]).split(",")[0]
<ipython-input-348-498ff371356a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["감독"][i] = 0
<ipython-input-348-498ff371356a>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["감독"][i] = "".join(List)
<ipython-input-348-498ff371356a>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

## 5. 개봉일

In [350]:
pd.options.display.float_format = '{:.5f}'.format

In [351]:
# 결측인 데이터에 1900-01-01로 만들기

df['개봉일'].fillna("1900-01-01", inplace = True)

In [352]:
# 개봉일 공백 제거

for i in range(len(df["개봉일"])):
    if type(df["개봉일"][i]) == float:
        df["개봉일"][i] = 0
    else:
        List = df["개봉일"][i].split(" ")
        df["개봉일"][i] = "".join(List)

<ipython-input-352-00e93f1db0c1>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["개봉일"][i] = "".join(List)


In [353]:
# 날짜 형식이 0000-00-00이 아닌 행의 인덱스 가져오기

index=[]

for i in range(len(df["개봉일"])):
    p= re.compile("\d\d\d\d-\d\d-\d\d")
    m= bool(p.match(str(df["개봉일"][i])))
    if m == False:
            index.append(i)

In [354]:
# 날짜형식이 yyyy-mm-dd아닌 데이터 수정

for j in range(len(df['개봉일'])):
    if len(df.iloc[j]['개봉일']) == 4:
        df['개봉일'][j] = df.iloc[j]['개봉일'] + "-01-01"

    elif len(df.iloc[j]['개봉일']) == 7:
        df['개봉일'][j] = df.iloc[j]['개봉일'] + "-01"
        
        
    elif  len(df.iloc[j]['개봉일']) >= 11:
        df['개봉일'][j] = df.iloc[j]['개봉일'][-10:]
        
    else: 
        continue

In [355]:
# 2021년 이전 데이터 삭제

df = df[df["개봉일"] >= "2021-01-01"]
df

,영화명,개봉일,누적매출액,누적관객수,스크린수,상영횟수,대표국적,배급사,등급,장르,감독,배우
0,블랙 위도우,2021-07-07,29846359280,2947549,2528,153914,미국,월트디즈니컴퍼니코리아유한책임회사,12세이상관람가,액션,케이트쇼트랜드,"[스칼렛요한슨, 플로렌스퓨, 레이첼와이즈]"
1,모가디슈,2021-07-28,23471251950,2442963,1688,119738,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,액션,류승완,"[김윤석, 조인성, 허준호]"
2,분노의 질주: 더 얼티메이트,2021-05-19,22059648060,2292413,2297,131856,미국,유니버설픽쳐스인터내셔널코리아(유),12세이상관람가,액션,저스틴린,"[빈디젤, 존시나, 성강]"
3,극장판 귀멸의 칼날: 무한열차편,2021-01-27,20653837290,2151572,900,151874,일본,워터홀컴퍼니(주),15세이상관람가,애니메이션,소토자키하루오,"[하나에나츠키, 키토아카리, 시모노히로]"
4,소울,2021-01-20,19034164180,2048137,2018,160725,미국,월트디즈니컴퍼니코리아유한책임회사,전체관람가,애니메이션,피트닥터,"[제이미폭스, 티나페이]"
...,...,...,...,...,...,...,...,...,...,...,...,...
2762,흥건한 며느리,2021-06-16,6000,1,1,1,일본,(주)영진크리에이티브,청소년관람불가,멜로로맨스,버드맨텟페이,[0]
2763,흥건한 새엄마,2021-04-26,6000,1,1,1,일본,(주)영진크리에이티브,청소년관람불가,멜로로맨스,사쿠라후부키,[0]
2764,흥건한 유부녀들,2021-06-24,6000,1,1,1,일본,(주)영진크리에이티브,청소년관람불가,멜로로맨스,사노B사쿠,[0]
2765,흥건한 제수씨:불타는 밤,2021-07-20,6000,1,1,1,일본,(주)영진크리에이티브,청소년관람불가,멜로로맨스,스이세이아카이,[사사키아키코]


In [356]:
# test data 파일생성
df.to_csv("test_data.csv", index = False, encoding = "utf-8-sig")

## 6. test DB

In [384]:
db_original = pd.read_csv('DB_data.csv', encoding = 'utf-8')

In [385]:
db_original = db_original[db_original["개봉일"] >= "2015-01-01"]
db_original = db_original[db_original["개봉일"] < "2021-01-01"]
db_original

,영화명,개봉일,누적관객수,배급사,감독,배우
1,미씨카: 더 네크로맨서,2017-11-02,60,(주)제이에이와이이엔터테인먼트,A.토드스미스,"애덤존슨,로버트제인,케빈소르보"
2,월드 워 핵전쟁의 서막,2019-03-07,17,주식회사풍경소리,A.K.스트롬,"모건브래들리,그레이엄빈센트,캠벨루셀"
3,울프라이징,2016-11-17,60,주식회사루믹스미디어,BC퍼트니,빌오버스트주니어
5,연쇄살인마의 이중생활,2020-06-18,7,주식회사마운틴픽쳐스,C.마틴페레라,"안나알렌,나초프레스네다,메르세데스카스트로"
6,몬스터 아일랜드,2017-10-27,1,주식회사레드아이스,C.블레이크에번든,존W.바우어스
...,...,...,...,...,...,...
21313,헤픈여자:많이 쌓였어요,2020-04-13,1,0,히이라기엔부,0
21314,화끈한 새엄마,2019-11-28,1,(주)영진크리에이티브,히이라기엔부,하나사키이안
21317,최면 섹스:멀티 오르가즘,2018-06-20,1,0,히토시이시카와,0
21322,16세의 사운드트랙,2020-07-02,413,(주)마노엔터테인먼트,힐러리셰익스피어,"제임스캘로웨이,스칼렛마셜"


In [386]:
db = pd.read_csv("테스트data.csv", thousands = ",", encoding = "utf-8")
db = db[["영화명", "개봉일", "누적관객수", "배급사", "감독", "배우"]]
db

,영화명,개봉일,누적관객수,배급사,감독,배우
0,블랙 위도우,2021-07-07,2947549,월트디즈니컴퍼니코리아 유한책임회사,케이트 쇼트랜드,"스칼렛 요한슨,플로렌스 퓨,레이첼 와이즈,데이빗 하버"
1,모가디슈,2021-07-28,2442963,롯데컬처웍스(주)롯데엔터테인먼트,류승완,"김윤석,조인성,허준호,구교환,김소진,정만식,김재화,박경혜,박명신,한철우,주보비,안세..."
2,분노의 질주: 더 얼티메이트,2021-05-19,2292413,유니버설픽쳐스인터내셔널 코리아(유),저스틴 린,"빈 디젤,존 시나,성 강,샤를리즈 테론,미셸 로드리게즈,조다나 브루스터,헬렌 미렌"
3,극장판 귀멸의 칼날: 무한열차편,2021-01-27,2151572,워터홀컴퍼니(주),소토자키 하루오,"하나에 나츠키,키토 아카리,시모노 히로,마츠오카 요시츠구,히노 사토시,히라카와 다이스케"
4,소울,2021-01-20,2048137,월트디즈니컴퍼니코리아 유한책임회사,피트 닥터,"제이미 폭스,티나 페이"
...,...,...,...,...,...,...
3010,하나가 되는 것은 더욱 커지는 일이다,NaN,21,시네마달,"김동원,문원립",NaN
3011,하늘의 황금마차,2014-09-04,2704,(주)영화사 진진,오멸,"문석범,김동호,양정원,이경준,킹스턴 루디스카,성낙원,손형식,장정인,조상원,조수범,한승진"
3012,항거:유관순 이야기,2019-02-27,1157953,롯데컬처웍스(주)롯데엔터테인먼트,조민호,"고아성,김새벽,김예은,정하담,류경수,최무성,하성광,류성민,권현진,김경록,김남진,박지..."
3013,홍길동,1967-01-21,68,NaN,신동헌,NaN


In [387]:
# 결측인 데이터에 1900-01-01로 만들기

pd.options.display.float_format = '{:.5f}'.format

db['개봉일'].fillna("1900-01-01", inplace = True)
db

,영화명,개봉일,누적관객수,배급사,감독,배우
0,블랙 위도우,2021-07-07,2947549,월트디즈니컴퍼니코리아 유한책임회사,케이트 쇼트랜드,"스칼렛 요한슨,플로렌스 퓨,레이첼 와이즈,데이빗 하버"
1,모가디슈,2021-07-28,2442963,롯데컬처웍스(주)롯데엔터테인먼트,류승완,"김윤석,조인성,허준호,구교환,김소진,정만식,김재화,박경혜,박명신,한철우,주보비,안세..."
2,분노의 질주: 더 얼티메이트,2021-05-19,2292413,유니버설픽쳐스인터내셔널 코리아(유),저스틴 린,"빈 디젤,존 시나,성 강,샤를리즈 테론,미셸 로드리게즈,조다나 브루스터,헬렌 미렌"
3,극장판 귀멸의 칼날: 무한열차편,2021-01-27,2151572,워터홀컴퍼니(주),소토자키 하루오,"하나에 나츠키,키토 아카리,시모노 히로,마츠오카 요시츠구,히노 사토시,히라카와 다이스케"
4,소울,2021-01-20,2048137,월트디즈니컴퍼니코리아 유한책임회사,피트 닥터,"제이미 폭스,티나 페이"
...,...,...,...,...,...,...
3010,하나가 되는 것은 더욱 커지는 일이다,1900-01-01,21,시네마달,"김동원,문원립",NaN
3011,하늘의 황금마차,2014-09-04,2704,(주)영화사 진진,오멸,"문석범,김동호,양정원,이경준,킹스턴 루디스카,성낙원,손형식,장정인,조상원,조수범,한승진"
3012,항거:유관순 이야기,2019-02-27,1157953,롯데컬처웍스(주)롯데엔터테인먼트,조민호,"고아성,김새벽,김예은,정하담,류경수,최무성,하성광,류성민,권현진,김경록,김남진,박지..."
3013,홍길동,1967-01-21,68,NaN,신동헌,NaN


In [388]:
# 개봉일 공백 제거

for i in range(len(db["개봉일"])):
    if type(db["개봉일"][i]) == float:
        db["개봉일"][i] = 0
    else:
        List = db["개봉일"][i].split(" ")
        db["개봉일"][i] = "".join(List)

<ipython-input-388-a6c0fdda6c34>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db["개봉일"][i] = "".join(List)


In [389]:
# 날짜 형식이 0000-00-00이 아닌 행의 인덱스 가져오기

index=[]

for i in range(len(db["개봉일"])):
    p= re.compile("\d\d\d\d-\d\d-\d\d")
    m= bool(p.match(str(db["개봉일"][i])))
    if m == False:
            index.append(i)

In [390]:
# 날짜형식이 yyyy-mm-dd아닌 데이터 수정

for j in range(len(db['개봉일'])):
    if len(str(db.iloc[j]['개봉일'])) == 4:
        db['개봉일'][j] = db.iloc[j]['개봉일'] + "-01-01"

    elif len(str(db.iloc[j]['개봉일'])) == 7:
        db['개봉일'][j] = db.iloc[j]['개봉일'] + "-01"
        
        
    elif  len(str(db.iloc[j]['개봉일'])) >= 11:
        db['개봉일'][j] = db.iloc[j]['개봉일'][-10:]
        
    else: 
        continue

In [391]:
db['배우'] = db['배우'].fillna(0)
db['배급사'] = db['배급사'].fillna(0)
db['감독'] = db['감독'].fillna(0)

In [401]:
db = db[['영화명', '개봉일', '누적관객수', '배급사', '감독', '배우']]
db = db[db["개봉일"] >= "2021-01-01"]
db

,영화명,개봉일,누적관객수,배급사,감독,배우
0,블랙 위도우,2021-07-07,2947549,월트디즈니컴퍼니코리아 유한책임회사,케이트 쇼트랜드,"스칼렛요한슨,플로렌스퓨,레이첼와이즈,데이빗하버"
1,모가디슈,2021-07-28,2442963,롯데컬처웍스(주)롯데엔터테인먼트,류승완,"김윤석,조인성,허준호,구교환,김소진,정만식,김재화,박경혜,박명신,한철우,주보비,안세..."
2,분노의 질주: 더 얼티메이트,2021-05-19,2292413,유니버설픽쳐스인터내셔널 코리아(유),저스틴 린,"빈디젤,존시나,성강,샤를리즈테론,미셸로드리게즈,조다나브루스터,헬렌미렌"
3,극장판 귀멸의 칼날: 무한열차편,2021-01-27,2151572,워터홀컴퍼니(주),소토자키 하루오,"하나에나츠키,키토아카리,시모노히로,마츠오카요시츠구,히노사토시,히라카와다이스케"
4,소울,2021-01-20,2048137,월트디즈니컴퍼니코리아 유한책임회사,피트 닥터,"제이미폭스,티나페이"
...,...,...,...,...,...,...
1154,흥건한 며느리,2021-06-16,1,(주)영진크리에이티브,버드맨 텟페이,0
1155,흥건한 새엄마,2021-04-26,1,(주)영진크리에이티브,사쿠라 후부키,0
1156,흥건한 유부녀들,2021-06-24,1,(주)영진크리에이티브,사노 B사쿠,0
1157,흥건한 제수씨:불타는 밤,2021-07-20,1,(주)영진크리에이티브,스이세이 아카이,사사키 아키코


In [399]:
db.reset_index(inplace=True)
db.drop("index", axis=1, inplace = True)

In [404]:
for i in range(len(db["배우"])):
    if db["배우"][i] == 0:
        db["배우"][i] = 0
    else:
        List = str(db["배우"][i]).split(" ")
        db["배우"][i] = "".join(List)

<ipython-input-404-cac149adfc5e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db["배우"][i] = "".join(List)
<ipython-input-404-cac149adfc5e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db["배우"][i] = 0


In [406]:
for i in range(len(db["배급사"])):
    if type(db["배급사"][i]) == float:
        db["배급사"][i] = 0
    else:
        List = str(db["배급사"][i]).split(" ")
        db["배급사"][i] = "".join(List)
        
for i in range(len(db["감독"])):
    if type(db["감독"][i]) == float:
        db["감독"][i] = 0
    else:
        List = str(db["감독"][i]).split(" ")
        db["감독"][i] = "".join(List)

<ipython-input-406-cdd855da34be>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db["배급사"][i] = "".join(List)
<ipython-input-406-cdd855da34be>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db["감독"][i] = "".join(List)


In [408]:
test_DB_data = pd.concat([db, db_original])
test_DB_data

,영화명,개봉일,누적관객수,배급사,감독,배우
0,블랙 위도우,2021-07-07,2947549,월트디즈니컴퍼니코리아유한책임회사,케이트쇼트랜드,"스칼렛요한슨,플로렌스퓨,레이첼와이즈,데이빗하버"
1,모가디슈,2021-07-28,2442963,롯데컬처웍스(주)롯데엔터테인먼트,류승완,"김윤석,조인성,허준호,구교환,김소진,정만식,김재화,박경혜,박명신,한철우,주보비,안세..."
2,분노의 질주: 더 얼티메이트,2021-05-19,2292413,유니버설픽쳐스인터내셔널코리아(유),저스틴린,"빈디젤,존시나,성강,샤를리즈테론,미셸로드리게즈,조다나브루스터,헬렌미렌"
3,극장판 귀멸의 칼날: 무한열차편,2021-01-27,2151572,워터홀컴퍼니(주),소토자키하루오,"하나에나츠키,키토아카리,시모노히로,마츠오카요시츠구,히노사토시,히라카와다이스케"
4,소울,2021-01-20,2048137,월트디즈니컴퍼니코리아유한책임회사,피트닥터,"제이미폭스,티나페이"
...,...,...,...,...,...,...
21313,헤픈여자:많이 쌓였어요,2020-04-13,1,0,히이라기엔부,0
21314,화끈한 새엄마,2019-11-28,1,(주)영진크리에이티브,히이라기엔부,하나사키이안
21317,최면 섹스:멀티 오르가즘,2018-06-20,1,0,히토시이시카와,0
21322,16세의 사운드트랙,2020-07-02,413,(주)마노엔터테인먼트,힐러리셰익스피어,"제임스캘로웨이,스칼렛마셜"


In [409]:
# test_DB_data 파일 생성
test_DB_data.to_csv("test_DB_data.csv", index = False, encoding = "utf-8-sig")

## 7. 배우 / 감독 / 배급사 숫자화

In [272]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
import progressbar

In [410]:
test_DB_data = pd.read_csv("test_DB_data.csv", index_col = False, encoding = "utf-8")
df = pd.read_csv("test_data.csv", index_col = False, encoding = "utf-8")

In [412]:
test_DB_data

,영화명,개봉일,누적관객수,배급사,감독,배우
0,블랙 위도우,2021-07-07,2947549,월트디즈니컴퍼니코리아유한책임회사,케이트쇼트랜드,"스칼렛요한슨,플로렌스퓨,레이첼와이즈,데이빗하버"
1,모가디슈,2021-07-28,2442963,롯데컬처웍스(주)롯데엔터테인먼트,류승완,"김윤석,조인성,허준호,구교환,김소진,정만식,김재화,박경혜,박명신,한철우,주보비,안세..."
2,분노의 질주: 더 얼티메이트,2021-05-19,2292413,유니버설픽쳐스인터내셔널코리아(유),저스틴린,"빈디젤,존시나,성강,샤를리즈테론,미셸로드리게즈,조다나브루스터,헬렌미렌"
3,극장판 귀멸의 칼날: 무한열차편,2021-01-27,2151572,워터홀컴퍼니(주),소토자키하루오,"하나에나츠키,키토아카리,시모노히로,마츠오카요시츠구,히노사토시,히라카와다이스케"
4,소울,2021-01-20,2048137,월트디즈니컴퍼니코리아유한책임회사,피트닥터,"제이미폭스,티나페이"
...,...,...,...,...,...,...
9030,헤픈여자:많이 쌓였어요,2020-04-13,1,0,히이라기엔부,0
9031,화끈한 새엄마,2019-11-28,1,(주)영진크리에이티브,히이라기엔부,하나사키이안
9032,최면 섹스:멀티 오르가즘,2018-06-20,1,0,히토시이시카와,0
9033,16세의 사운드트랙,2020-07-02,413,(주)마노엔터테인먼트,힐러리셰익스피어,"제임스캘로웨이,스칼렛마셜"


In [318]:
test_DB_data.reset_index(inplace=True)
test_DB_data.drop("index", axis=1, inplace = True)

In [413]:
# 문자형으로 들어있는 개봉일을 날짜형으로 변경

df["개봉일"]=  pd.to_datetime(df["개봉일"], format="%Y-%m-%d")
test_DB_data["개봉일"]=  pd.to_datetime(test_DB_data["개봉일"], format="%Y-%m-%d")

In [320]:
df = df.astype({'배우': 'str'})
test_DB_data = test_DB_data.astype({'배우': 'str'})

In [414]:
# 배우 -> 관객수 데이터로 치환

start = time.time()
bar = progressbar.ProgressBar()
df[["배우1","배우2","배우3"]] = 0

for nu in bar(range(len(df["배우"]))):
    if df["배우"][nu].strip("[]").split(", ")[0].strip("'") == 0:
        df["배우1"][nu] = 0
        
    else:
        act_list = df.iloc[nu]["배우"].strip("[]").split(", ")
        for numb in range(len(act_list)):
            count=0
            audi = 0
            for index in range(len(test_DB_data["배우"])):  
                if act_list[numb].strip("'") in test_DB_data["배우"][index] and (df.iloc[nu]["개봉일"] > test_DB_data["개봉일"][index]) & (df.iloc[nu]["개봉일"]- relativedelta(years=6) < test_DB_data["개봉일"][index]):
                        count+=1
                        audi += test_DB_data["누적관객수"][index]
            if count != 0:
                audi = audi / count
                df["배우" + str(numb + 1)][nu] = audi
                
end = time.time()
print(end - start) 

<ipython-input-414-67ed3dc19593>:22: SettingWithCopyWarning:                  |
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["배우" + str(numb + 1)][nu] = audi


2532.906173467636


In [415]:
# 감독 -> 관객수 데이터로 치환

start = time.time()
bar = progressbar.ProgressBar()
df["감독값"]= 0

for nu in bar(range(len(df["감독"]))):
    if df["감독"][nu] =='0':
        df["감독값"][nu] = 0 
        
    else:
        count=0
        audi = 0
        for index in range(len(test_DB_data["감독"])):  
            if df["감독"][nu] in test_DB_data["감독"][index] and (df.iloc[nu]["개봉일"] > test_DB_data["개봉일"][index]) & (df.iloc[nu]["개봉일"]- relativedelta(years=6) < test_DB_data["개봉일"][index]):
                    count+=1
                    audi += test_DB_data["누적관객수"][index]
        if count != 0:
            audi = audi/count
            df["감독값"][nu] = audi
            
end = time.time()
print(end - start) 

<ipython-input-415-72fd0527693f>:20: SettingWithCopyWarning:                  |
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["감독값"][nu] = audi
<ipython-input-415-72fd0527693f>:9: SettingWithCopyWarning:                   |
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["감독값"][nu] = 0


250.55055737495422


In [416]:
# 배급사 -> 관객수 데이터로 치환

start = time.time()
bar = progressbar.ProgressBar()
df["배급사값"]= 0

for nu in bar(range(len(df["배급사"]))):
    if df["배급사"][nu] == 0:
        continue
        
    else:
        count=0
        audi = 0
        for index in range(len(test_DB_data["배급사"])):  
            if df["배급사"][nu] in test_DB_data["배급사"][index] and (df.iloc[nu]["개봉일"] > test_DB_data["개봉일"][index]) & (df.iloc[nu]["개봉일"]- relativedelta(years=6) < test_DB_data["개봉일"][index]):
                    count+=1
                    audi += test_DB_data["누적관객수"][index]
        if count != 0:
            audi = audi/count
            df["배급사값"][nu] = audi
            
end = time.time()
print(end - start) 

<ipython-input-416-72cf44f02970>:20: SettingWithCopyWarning:                  |
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["배급사값"][nu] = audi


550.1465032100677


In [421]:
# 배우합 만들기

df["배우합"]= 0

for i in range(len(df)):
    if len(df["배우"][i].strip("[]").split(", "))== 3:
        df["배우합"][i] = df.iloc[i]["배우1"]+ df.iloc[i]["배우2"] +df.iloc[i]["배우3"]

    elif len(df["배우"][i].strip("[]").split(", "))== 2:
        df["배우합"][i] = df["배우1"][i]+ df["배우2"][i]

    else:
        df["배우합"][i] = df["배우1"][i]

<ipython-input-421-4572ca5a22d2>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["배우합"][i] = df.iloc[i]["배우1"]+ df.iloc[i]["배우2"] +df.iloc[i]["배우3"]
<ipython-input-421-4572ca5a22d2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["배우합"][i] = df["배우1"][i]+ df["배우2"][i]
<ipython-input-421-4572ca5a22d2>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["배우합"][i] = df["배우1"][i]


In [426]:
df

,영화명,개봉일,누적매출액,누적관객수,스크린수,상영횟수,대표국적,배급사,등급,장르,...,이탈리아,일본,중국,캐나다,프랑스,한국,12세이상관람가,15세이상관람가,전체관람가,청소년관람불가
0,블랙 위도우,2021-07-07,29846359280,2947549,2528,153914,미국,월트디즈니컴퍼니코리아유한책임회사,12세이상관람가,액션,...,0,0,0,0,0,0,1,0,0,0
1,모가디슈,2021-07-28,23471251950,2442963,1688,119738,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,액션,...,0,0,0,0,0,1,0,1,0,0
2,분노의 질주: 더 얼티메이트,2021-05-19,22059648060,2292413,2297,131856,미국,유니버설픽쳐스인터내셔널코리아(유),12세이상관람가,액션,...,0,0,0,0,0,0,1,0,0,0
3,극장판 귀멸의 칼날: 무한열차편,2021-01-27,20653837290,2151572,900,151874,일본,워터홀컴퍼니(주),15세이상관람가,애니메이션,...,0,1,0,0,0,0,0,1,0,0
4,소울,2021-01-20,19034164180,2048137,2018,160725,미국,월트디즈니컴퍼니코리아유한책임회사,전체관람가,애니메이션,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1147,흥건한 며느리,2021-06-16,6000,1,1,1,일본,(주)영진크리에이티브,청소년관람불가,멜로로맨스,...,0,1,0,0,0,0,0,0,0,1
1148,흥건한 새엄마,2021-04-26,6000,1,1,1,일본,(주)영진크리에이티브,청소년관람불가,멜로로맨스,...,0,1,0,0,0,0,0,0,0,1
1149,흥건한 유부녀들,2021-06-24,6000,1,1,1,일본,(주)영진크리에이티브,청소년관람불가,멜로로맨스,...,0,1,0,0,0,0,0,0,0,1
1150,흥건한 제수씨:불타는 밤,2021-07-20,6000,1,1,1,일본,(주)영진크리에이티브,청소년관람불가,멜로로맨스,...,0,1,0,0,0,0,0,0,0,1


## 8. 장르 / 대표국적 / 등급 dummy

In [424]:
dummy_genre = pd.get_dummies(df["장르"])
dummy_nation = pd.get_dummies(df["대표국적"])
dummy_level = pd.get_dummies(df["등급"])

In [425]:
df = df.join(dummy_genre)
df = df.join(dummy_nation)
df = df.join(dummy_level)

In [429]:
df.to_csv("test_data.csv", index = False, encoding = "utf-8-sig")

## 9. 평점 / 평점평가자수 / 리뷰평가자수

In [448]:
review = pd.read_excel("test_review.xlsx", thousands = ",")
review = review[['개봉일','평점', '리뷰수', '리뷰개수']]
review

,개봉일,평점,리뷰수,리뷰개수
0,2021-07-07,8.50000,7061,194
1,2021-07-28,8.22000,11249,115
2,2021-05-19,7.46000,5950,137
3,2021-01-27,9.28000,12774,94
4,2021-01-20,9.31000,9894,275
...,...,...,...,...
3010,NaN,0.00000,0,0
3011,2014-09-04,7.08000,48,30
3012,2019-02-27,8.65000,7443,220
3013,1967-01-21,7.75000,9870,4


In [450]:
pd.options.display.float_format = '{:.5f}'.format
#review["개봉일"]=  pd.to_datetime(review["개봉일"], format="%Y-%m-%d")

In [451]:
# 결측인 데이터에 1900-01-01로 만들기

review['개봉일'].fillna("1900-01-01", inplace = True)

In [452]:
# 개봉일 공백 제거

for i in range(len(review["개봉일"])):
    if type(review["개봉일"][i]) == float:
        review["개봉일"][i] = 0
    else:
        List = review["개봉일"][i].split(" ")
        review["개봉일"][i] = "".join(List)

<ipython-input-452-081705d73975>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review["개봉일"][i] = "".join(List)


In [454]:
# 날짜 형식이 0000-00-00이 아닌 행의 인덱스 가져오기

index=[]

for i in range(len(review["개봉일"])):
    p= re.compile("\d\d\d\d-\d\d-\d\d")
    m= bool(p.match(str(review["개봉일"][i])))
    if m == False:
            index.append(i)

In [455]:
# 날짜형식이 yyyy-mm-dd아닌 데이터 수정

for j in range(len(review['개봉일'])):
    if len(review.iloc[j]['개봉일']) == 4:
        review['개봉일'][j] = review.iloc[j]['개봉일'] + "-01-01"

    elif len(review.iloc[j]['개봉일']) == 7:
        review['개봉일'][j] = review.iloc[j]['개봉일'] + "-01"
        
        
    elif  len(review.iloc[j]['개봉일']) >= 11:
        review['개봉일'][j] = review.iloc[j]['개봉일'][-10:]
        
    else: 
        continue

In [456]:
# 2021년 이전 데이터 삭제

review = review[review["개봉일"] >= "2021-01-01"]

,개봉일,평점,리뷰수,리뷰개수
0,2021-07-07,8.50000,7061,194
1,2021-07-28,8.22000,11249,115
2,2021-05-19,7.46000,5950,137
3,2021-01-27,9.28000,12774,94
4,2021-01-20,9.31000,9894,275
...,...,...,...,...
2945,2021-06-16,0.00000,0,0
2946,2021-04-26,0.00000,0,0
2947,2021-06-24,0.00000,0,0
2948,2021-07-20,0.00000,0,0


In [457]:
review.reset_index(inplace=True)
review.drop("index", axis=1, inplace = True)
review

C:\Users\joong\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,개봉일,평점,리뷰수,리뷰개수
0,2021-07-07,8.50000,7061,194
1,2021-07-28,8.22000,11249,115
2,2021-05-19,7.46000,5950,137
3,2021-01-27,9.28000,12774,94
4,2021-01-20,9.31000,9894,275
...,...,...,...,...
1154,2021-06-16,0.00000,0,0
1155,2021-04-26,0.00000,0,0
1156,2021-06-24,0.00000,0,0
1157,2021-07-20,0.00000,0,0


In [458]:
df["평점"] = review["평점"]
df["평점평가자수"] = review["리뷰수"]
df["리뷰평가자수"] = review["리뷰개수"]

In [459]:
df

,영화명,개봉일,누적매출액,누적관객수,스크린수,상영횟수,대표국적,배급사,등급,장르,...,캐나다,프랑스,한국,12세이상관람가,15세이상관람가,전체관람가,청소년관람불가,평점,평점평가자수,리뷰평가자수
0,블랙 위도우,2021-07-07,29846359280,2947549,2528,153914,미국,월트디즈니컴퍼니코리아유한책임회사,12세이상관람가,액션,...,0,0,0,1,0,0,0,8.50000,7061,194
1,모가디슈,2021-07-28,23471251950,2442963,1688,119738,한국,롯데컬처웍스(주)롯데엔터테인먼트,15세이상관람가,액션,...,0,0,1,0,1,0,0,8.22000,11249,115
2,분노의 질주: 더 얼티메이트,2021-05-19,22059648060,2292413,2297,131856,미국,유니버설픽쳐스인터내셔널코리아(유),12세이상관람가,액션,...,0,0,0,1,0,0,0,7.46000,5950,137
3,극장판 귀멸의 칼날: 무한열차편,2021-01-27,20653837290,2151572,900,151874,일본,워터홀컴퍼니(주),15세이상관람가,애니메이션,...,0,0,0,0,1,0,0,9.28000,12774,94
4,소울,2021-01-20,19034164180,2048137,2018,160725,미국,월트디즈니컴퍼니코리아유한책임회사,전체관람가,애니메이션,...,0,0,0,0,0,1,0,9.31000,9894,275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1147,흥건한 며느리,2021-06-16,6000,1,1,1,일본,(주)영진크리에이티브,청소년관람불가,멜로로맨스,...,0,0,0,0,0,0,1,0.00000,0,0
1148,흥건한 새엄마,2021-04-26,6000,1,1,1,일본,(주)영진크리에이티브,청소년관람불가,멜로로맨스,...,0,0,0,0,0,0,1,6.00000,2,0
1149,흥건한 유부녀들,2021-06-24,6000,1,1,1,일본,(주)영진크리에이티브,청소년관람불가,멜로로맨스,...,0,0,0,0,0,0,1,0.00000,0,0
1150,흥건한 제수씨:불타는 밤,2021-07-20,6000,1,1,1,일본,(주)영진크리에이티브,청소년관람불가,멜로로맨스,...,0,0,0,0,0,0,1,0.00000,0,0


## 10. 열추가 / 삭제
+ 모험 / 느와르/ 무협 / 서사 / 장르없음 / 등급없음 추가
+ 대표국가 더미 삭제

In [17]:
df = pd.read_csv("test_data.csv", thousands = ",", encoding = "utf-8")

In [18]:
df["모험"] = 0
df["느와르"] = 0
df["무협"] = 0
df["서사"] = 0
df["등급없음"] = 0
df["장르없음"] = 0

In [19]:
del df["기타"]
del df["독일"]
del df["미국"]
del df["스페인"]
del df["영국"]
del df["이탈리아"]
del df["일본"]
del df["중국"]
del df["캐나다"]
del df["프랑스"]
del df["한국"]

In [21]:
del df["배우"]
del df["배우1"]
del df["배우2"]
del df["배우3"]

In [22]:
df.columns

Index(['영화명', '개봉일', '누적매출액', '누적관객수', '스크린수', '상영횟수', '대표국적', '배급사', '등급',
       '장르', '감독', '감독값', '배급사값', '배우합', 'SF', '가족', '공연', '공포', '다큐멘터리',
       '드라마', '멜로로맨스', '뮤지컬', '미스터리', '범죄', '사극', '서부', '스릴러', '애니메이션', '액션',
       '어드벤처', '에로', '전쟁', '코미디', '판타지', '12세이상관람가', '15세이상관람가', '전체관람가',
       '청소년관람불가', '평점', '평점평가자수', '리뷰평가자수', '모험', '느와르', '무협', '서사', '등급없음',
       '장르없음'],
      dtype='object')

In [23]:
df.to_csv("test_data.csv", index = False, encoding = "utf-8-sig")